In [ ]:
# installs
! pip3 install --user kfp google-cloud-pipeline-components --upgrade --quiet
! pip3 install --upgrade google-cloud-aiplatform --user -q

! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

In [ ]:
### Now restart the notebook kernel
# in the top toolbar:  "Kernel" -> "Restart Kernel"

In [ ]:
#####################################################################
#
# setup
#
#####################################################################

In [2]:
# specify parameters
PROJECT_ID = "your-project-id"
REGION = "us-central1"
BUCKET_NAME = f"bkt-{PROJECT_ID}-vpipelines"
BUCKET_PATH = f"gs://{BUCKET_NAME}"
PIPELINE_ROOT = f"{BUCKET_PATH}/pipeline_root"
PIPELINE_DATA = f"{BUCKET_PATH}/data"
TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

In [ ]:
# imports for this notebook to run
import sys
from datetime import datetime
from typing import NamedTuple

from google.cloud import aiplatform as vertex

import kfp
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output, OutputPath, component)

In [ ]:
#####################################################################
#
# create needed resources
#
#####################################################################

In [ ]:
# check if bucket exists
def check_bucket_existence(PROJECT_ID, BUCKET_PATH):
    from google.cloud import storage
    from google.cloud.storage import Bucket

    client = storage.Client(project=PROJECT_ID)
    exists = Bucket(client, BUCKET_PATH).exists()
    return exists

# create bucket if needed
if check_bucket_existence(PROJECT_ID, BUCKET_NAME) == False:
    ! gsutil mb -c standard -l $REGION -b on -p $PROJECT_ID $BUCKET_PATH
    ! gsutil ls
else:
    print(f"Bucket {BUCKET_NAME} already exists")

In [ ]:
#####################################################################
# BEGIN vertex pipelines
#####################################################################

In [ ]:
#####################################################################
#
# create individual pipeline components, then specify the pipeline
#
#####################################################################

In [24]:
# create a custom component
@component(base_image="python:3.9", output_component_file="step_1.yaml")

#specify inputs, outputs, and component functionality
def step_1(mystring: str # inputs and their expected types
          ) -> NamedTuple("Outputs", [ ("mystring", str) ] #outputs and their expected types
                         ):    
    # component functionality
    mystring = mystring + ' world'
    # any outputs to return
    return (mystring,) # comma required bc an iterable is expected

In [25]:
# define a pipeline
@dsl.pipeline(name="my-pipeline-name", description="my pipeline description")

# specify all the inputs the pipeline needs to run
def my_pipeline(mystr: str, project_id: str = PROJECT_ID):
    
    # specify the nodes in the pipeline
    step_1_task = step_1(mystr)

In [ ]:
#####################################################################
#
# compile and run the pipeline
#
#####################################################################

In [26]:
# compile the pipeline
my_package_path = 'my_vertex_pipeline_specification_file.json'
compiler.Compiler().compile(pipeline_func=my_pipeline, package_path=my_package_path)

In [34]:
# runtime parameters to pass to pipeline
pipeline_params = {
    "project_id": PROJECT_ID,
    "mystr": "hello"
}

# run the pipeline
vertex.init(project=PROJECT_ID)

job = vertex.PipelineJob(
    display_name = "my-pipeline-job-name",
    template_path = my_package_path,
    pipeline_root = PIPELINE_ROOT,
    parameter_values = pipeline_params,
    enable_caching = False
)

job.submit()

Creating PipelineJob
PipelineJob created. Resource name: projects/720376660491/locations/us-central1/pipelineJobs/my-pipeline-name-20220929192449
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/720376660491/locations/us-central1/pipelineJobs/my-pipeline-name-20220929192449')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/my-pipeline-name-20220929192449?project=720376660491
